In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import logging

# 자체 제작 유틸리티 모듈 임포트
import data_utils

# 로깅 설정
# data_utils 모듈의 setup_logging 함수를 호출하여 로깅을 설정합니다.
# 기존 핸들러 제거 및 basicConfig 설정은 이 함수 안에서 처리됩니다.
data_utils.setup_logging()
logging.info("필수 라이브러리 및 data_utils 모듈 임포트, 로깅 설정 완료")

2025-04-19 00:19:45,203 - INFO - 로깅 설정 완료
2025-04-19 00:19:45,203 - INFO - 필수 라이브러리 및 data_utils 모듈 임포트, 로깅 설정 완료


In [ ]:
# 직종 구분 선택 -> 사이트마다 다르게 설정할 것
job_category = "total" # "total" # "backend" # "frontend"

if job_category == "total":
    job_category_url = "?jobGroup=DEVELOPER"
elif job_category == "backend":
    job_category_url = "?job=BACKEND_DEVELOPER&jobGroup=DEVELOPER"
elif job_category == "frontend":
    job_category_url = "?job=FRONTEND_DEVELOPER&jobGroup=DEVELOPER"
else:
    job_category == "total"
    job_category_url = "?jobGroup=DEVELOPER"

scraped_data = []
start_page_no = 1

base_url = "https://www.rallit.com"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("스크래핑 설정 완료.")

2025-04-19 00:19:45,217 - INFO - 스크래핑 설정 완료.


In [3]:
# 셀3
# CSS 선택자를 이용하여 soup 객체에서 데이터를 추출하는 함수.
# 이 함수는 현재 스크립트(Notebook)에서 사용됩니다.
def extract_data(soup, selector, attribute=None, is_list=False, default="N/A"):
    """
    주어진 CSS 선택자를 사용하여 BeautifulSoup 객체에서 데이터를 추출합니다.
    Args:
        soup (BeautifulSoup): 데이터를 추출할 BeautifulSoup 객체.
        selector (str): CSS 선택자.
        attribute (str, optional): 추출할 속성. None이면 텍스트 추출. Defaults to None.
        is_list (bool, optional): 여러 개의 결과를 반환할지 여부. Defaults to False.
        default (str, optional): 찾지 못했을 경우 반환할 기본값. Defaults to "N/A".

    Returns:
        str or list: 추출된 텍스트 또는 속성 값 (is_list=True인 경우 리스트).
    """
    try:
        if is_list:
            elements = soup.select(selector)
            if attribute:
                return [element.get(attribute, default).strip() for element in elements]
            else:
                return [element.get_text(strip=True) for element in elements]
        else:
            element = soup.select_one(selector)
            if element:
                if attribute:
                    return element.get(attribute, default).strip()
                else:
                    return element.get_text(strip=True)
            else:
                # print(f"Warning: Could not find element with selector '{selector}'.") # 로깅으로 대체 권장
                logging.warning(f"Could not find element with selector '{selector}'.")
                return default
    except Exception as e:
        # print(f"Error extracting data with selector '{selector}': {e}") # 로깅으로 대체 권장
        logging.error(f"Error extracting data with selector '{selector}': {e}", exc_info=True)
        return default

In [4]:
# 셀4
# 실제 스크래핑 루프 실행
page_no = start_page_no
while True: # 무한 루프 시작 (종료 조건은 루프 내에 있음)
    # 대상 URL 구성
    list_url = f"{base_url}{job_category_url}&pageNumber={page_no}"
    logging.info(f"Scraping page: {list_url}")

    try:
        # 1. 채용 공고 목록 페이지 가져오기
        # data_utils.DEFAULT_HEADERS 상수를 사용합니다.
        response = requests.get(list_url, headers=data_utils.DEFAULT_HEADERS)
        response.raise_for_status() # HTTP 오류 발생 시 예외 발생
        soup = BeautifulSoup(response.text, 'html.parser')

        # 2. 각 채용 공고 컨테이너(article) 찾기
        job_postings = soup.find_all('article')

        # *** 동적 로딩 확인 및 루프 종료 조건 ***
        # 페이지에 공고가 없거나, 개발자 공고 제목 요소를 찾지 못하면 종료
        if not job_postings or not soup.select_one('h3.summary__title.css-5g43jj'):
            if not job_postings:
                logging.info(f"Page {page_no}: 채용 공고 article 태그를 찾을 수 없습니다. 스크래핑을 종료합니다.")
                print("\n채용 공고 article 태그를 찾을 수 없습니다. 스크래핑을 종료합니다.")
            else: # job_postings는 있지만, 특정 css selector가 없는 경우
                logging.info(f"Page {page_no}: 더 이상 개발자 공고 제목(h3.summary__title.css-5g43jj)을 찾을 수 없습니다. 스크래핑을 종료합니다.")
                print("\n더 이상 개발자 공고 제목을 찾을 수 없습니다. 스크래핑을 종료합니다.")
            break # 공고가 없거나 종료 조건 만족 시 루프 종료
        else:
            logging.info(f"Found {len(job_postings)} job postings on page {page_no}.")
            print(f"Found {len(job_postings)} job postings on page {page_no}.")


        # 3. 각 공고별 정보 추출
        for item in job_postings:
            company_name = "N/A"
            position = "N/A"
            skill = "N/A"

            # 각 채용 공고 article 안의 div.css-vjt50z 컨테이너 찾기
            job_info_container = item.find('div', class_='css-vjt50z')

            if job_info_container:
                # 3-1. 회사명 추출
                company_name_element = job_info_container.find('p', class_='summary__company-name css-x5ccem')
                if company_name_element:
                    company_name = company_name_element.get_text(strip=True)

                # 3-2. 공고 제목 추출
                position_element = job_info_container.find('h3', class_='summary__title css-5g43jj')
                if position_element:
                    position = position_element.get_text(strip=True)

                # 3-3. 상세 페이지 링크 추출 (모든 skill 추출)
                skill_elements = job_info_container.find_all('p', class_='css-13kyeyo')
                skills_list = [skill_el.get_text(strip=True) for skill_el in skill_elements] # 변수명 충돌 피함
                skill = ', '.join(skills_list) if skills_list else "N/A"

                # --- 여기에 filtering_utils 모듈의 필터링 함수 적용 ---
                filtered_skill = data_utils.filter_skill_data(skill)
                # --- 필터링 완료 ---

                # 3-5. 결과 저장
                scraped_data.append({
                    'company': company_name,
                    'position': position,
                    'skill': filtered_skill,
                })
                # 상세 로그는 너무 길 수 있으니 필요에 따라 주석 처리하거나 레벨 조정
                # logging.debug("    --- Scraped Data ---")
                # logging.debug(f"    company: {company_name}")
                # logging.debug(f"    position: {position}")
                # logging.debug(f"    skill: {filtered_skill[:100]}..." if filtered_skill and len(filtered_skill) > 100 else filtered_skill if filtered_skill else "N/A")
                # logging.debug("    ----------------------")
            else:
                logging.warning("    --- No job info container found for an article element ---")
                # print("    --- No job info container found ---") # 로깅으로 대체 권장

        page_no += 1 # 다음 페이지로 이동
        print(f"Current page number: {page_no}")
        time.sleep(1) # 페이지 간 딜레이 (필수)

    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching list page {list_url}: {e}", exc_info=True)
        print(f"\nError fetching list page {list_url}: {e}")
        break # 네트워크 또는 HTTP 오류 발생 시 루프 종료
    except Exception as e:
        logging.error(f"An unexpected error occurred during scraping: {e}", exc_info=True)
        print(f"\nAn unexpected error occurred: {e}")
        break

logging.info("스크래핑 루프 종료.")

2025-04-19 00:19:45,246 - INFO - Scraping page: https://www.rallit.com?jobGroup=DEVELOPER&pageNumber=1
2025-04-19 00:19:45,512 - INFO - Found 20 job postings on page 1.


Found 20 job postings on page 1.
Current page number: 2


2025-04-19 00:19:46,516 - INFO - Scraping page: https://www.rallit.com?jobGroup=DEVELOPER&pageNumber=2
2025-04-19 00:19:46,686 - INFO - Found 20 job postings on page 2.


Found 20 job postings on page 2.
Current page number: 3


2025-04-19 00:19:47,691 - INFO - Scraping page: https://www.rallit.com?jobGroup=DEVELOPER&pageNumber=3
2025-04-19 00:19:47,948 - INFO - Found 20 job postings on page 3.


...

Found 20 job postings on page 15.
Current page number: 16


2025-04-19 00:20:03,290 - INFO - Scraping page: https://www.rallit.com?jobGroup=DEVELOPER&pageNumber=16
2025-04-19 00:20:03,445 - INFO - Found 12 job postings on page 16.


Found 12 job postings on page 16.
Current page number: 17


2025-04-19 00:20:04,448 - INFO - Scraping page: https://www.rallit.com?jobGroup=DEVELOPER&pageNumber=17
2025-04-19 00:20:04,571 - INFO - Page 17: 채용 공고 article 태그를 찾을 수 없습니다. 스크래핑을 종료합니다.
2025-04-19 00:20:04,571 - INFO - 스크래핑 루프 종료.



채용 공고 article 태그를 찾을 수 없습니다. 스크래핑을 종료합니다.


In [5]:
# 셀5
# CSV 파일 저장
# CSV 파일 이름을 설정합니다. 이 이름은 이 Notebook 실행에 고유합니다.
data_folder = 'data'
# filename = f'data_rallit_{job_category}.csv'
filename = 'temp.csv'

# data_utils 모듈의 save_data_to_csv 함수를 사용하여 데이터를 저장합니다.
# 폴더 생성, DataFrame 변환 및 CSV 저장 로직은 함수 안에서 처리됩니다.
saved_filepath = data_utils.save_data_to_csv(scraped_data, filename, folder=data_folder)

if saved_filepath:
    logging.info(f"데이터 저장 과정 완료. 파일 경로: {saved_filepath}")
else:
    logging.warning("데이터 저장에 실패했거나 저장할 데이터가 없습니다.")

2025-04-19 00:20:04,580 - INFO - 수집된 데이터를 DataFrame으로 변환 중...
2025-04-19 00:20:04,582 - INFO - 'data' 폴더가 이미 존재합니다.
2025-04-19 00:20:04,590 - INFO - DataFrame이 'data\temp.csv'으로 성공적으로 저장되었습니다.
2025-04-19 00:20:04,591 - INFO - 데이터 저장 과정 완료. 파일 경로: data\temp.csv



파일 저장 완료: data\temp.csv


In [6]:
# 셀6
# 저장된 CSV 파일을 읽어와 DataFrame으로 출력 (확인용)
# 저장된 파일 경로를 사용하거나, filename과 data_folder를 조합하여 사용합니다.
# save_data_to_csv 함수가 성공 시 파일 경로를 반환하므로 그것을 사용합니다.
if saved_filepath:
    # data_utils 모듈의 load_data_from_csv 함수를 사용하여 파일을 읽어옵니다.
    df_read = data_utils.load_data_from_csv(saved_filepath)

    if df_read is not None:
        logging.info("저장된 CSV 파일 DataFrame 출력.")
        print("\n--- 로드된 데이터프레임 (처음 5행) ---")
        print(df_read.head())
        print(f"\n총 로드된 행 수: {len(df_read)}")
        print("------------------------------------")
    else:
        logging.warning("CSV 파일을 로드하지 못했습니다.")
else:
    logging.warning("저장된 파일 경로가 없어 CSV 로드 테스트를 건너뜁니다.")

2025-04-19 00:20:04,620 - INFO - 'data\temp.csv' 파일에서 DataFrame 로드 성공.
2025-04-19 00:20:04,620 - INFO - 저장된 CSV 파일 DataFrame 출력.



--- 로드된 데이터프레임 (처음 5행) ---
    company                                    position  \
0      오비맥주  [ABInBev Korea 오비맥주] BEES Product Engineer   
1       와디즈            [와디즈] 자바 백엔드 개발 (병역특례/전문연구요원/전직)   
2  이지식스(엠블)                          [TADA] QA Engineer   
3  CLASS101                                  소프트웨어 엔지니어   
4    (주)야나두                     [야나두(야핏)] 안드로이드(AOS) 개발   

                                               skill  
0                                             Access  
1            Java, Spring, Boot, REST, API, AWS, Git  
2                                       JIRA, Github  
3  Java, Kotlin, Spring, Boot, MongoDB, AWS, Kube...  
4                        Kotlin, Java, MVVM, Jetpack  

총 로드된 행 수: 312
------------------------------------
